In [2]:
#Import the necessary libraries and choose dataset
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
#Streamline switching models
def build_model(use_big_dense=False):
  return keras.Sequential([
    layers.Conv2D(32, 3, activation="relu", input_shape=(28, 28 , 1)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(4096 if use_big_dense else 128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(10, activation="softmax"),
  ])

  #Set Parameters
  use_fashion = True
  use_big_dense = False
  batch_size = 128
  epochs = 5

#Run experiment (dataset + model size)
def run_experiment(use_fashion=True, use_big_dense=False, epochs=5, batch_size=128):

  #Pick dataset
  (x_train, y_train), (x_test, y_test) = (
      keras.datasets.fashion_mnist if use_fashion else keras.datasets.mnist
  ).load_data()

  x_train = x_train.astype("float32") / 255.0
  x_test = x_test.astype("float32") / 255.0
  x_train = x_train[..., None]
  x_test = x_test[..., None]

  #Build model and compile
  model = build_model(use_big_dense)
  model.compile(
      optimizer="adam",
      loss="sparse_categorical_crossentropy",
      metrics=["accuracy"]
  )

  start_time = time.time()
  history = model.fit(
      x_train,
      y_train,
      batch_size=batch_size,
      epochs=epochs,
      validation_split=0.1,
      verbose=2
  )
  train_time = time.time() - start_time

  #Evaluate on the test set
  test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

#Run all combinations and return data
results = []

results.append(run_experiment(use_fashion=False, use_big_dense=False))#MNIST
results.append(run_experiment(use_fashion=True, use_big_dense=False))#Fashion

results.append(run_experiment(use_fashion=False, use_big_dense=True))#MNIST Dense
results.append(run_experiment(use_fashion=True, use_big_dense=True))#Fashion Dense

#Print Results
results("\n--- Accuracy & Time Comparison ---")
for r in results:
    print(f"{r['dataset']:13s} | {r['model']:10s} "
          f"epochs={r['epochs']:2d} | time={r['train_time_sec']:6.2f}s |"
          f"acc={r['test_acc']:.4f} | loss={r['test_loss']:.4f}")



/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
422/422 - 44s - 103ms/step - accuracy: 0.9062 - loss: 0.3080 - val_accuracy: 0.9843 - val_loss: 0.0597
Epoch 2/5
422/422 - 84s - 200ms/step - accuracy: 0.9703 - loss: 0.1003 - val_accuracy: 0.9893 - val_loss: 0.0424
Epoch 3/5
422/422 - 79s - 188ms/step - accuracy: 0.9786 - loss: 0.0724 - val_accuracy: 0.9892 - val_loss: 0.0371
Epoch 4/5
422/422 - 42s - 101ms/step - accuracy: 0.9823 - loss: 0.0587 - val_accuracy: 0.9900 - val_loss: 0.0363
Epoch 5/5
422/422 - 80s - 190ms/step - accuracy: 0.9841 - loss: 0.0512 - val_accuracy: 0.9898 - val_loss: 0.0367
Epoch 1/5
422/422 - 47s - 110ms/step - accuracy: 0.7553 - loss: 0.6738 - val_accuracy: 0.8498 - val_loss: 0.4106
Epoch 2/5
422/422 - 44s - 105ms/step - accuracy: 0.8472 - loss: 0.4308 - val_accuracy: 0.8767 - val_loss: 0.3385
Epoch 3/5
422/422 - 83s - 196ms/step - accuracy: 0.8667 - loss: 0.3734 - val_accuracy: 0.8887 - val_loss: 0.3095
Epoch 4/5
422/422 - 44s - 104ms/step - accuracy: 0.8774 - loss: 0.3407 - val_accuracy: 0.8902 - 

In [ ]:
#Return Summary
print({
    "dataset": "fashion" if use_fashion else "mnist",
    "model_size": "dense4096" if use_big_dense else "dense128",
    "epochs": epochs,
    "train_time_sec": round(train_time, 2),
    "test_acc": round(test_acc, 4),
    "test_loss": round(test_loss, 4)
})
